In [19]:
from pyspark import SparkContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.ml.linalg import DenseVector
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.evaluation import MulticlassMetrics

%matplotlib inline

In [20]:
hr_kaggle = sc.textFile('HR_comma_sep.csv')

In [21]:
hr_kaggle = hr_kaggle.map(lambda x: x.split(','))

In [22]:
hr_kaggle_final = hr_kaggle.map(lambda x: (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9]))

In [23]:
hr_kaggle_final.take(5)

[('satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years',
  'sales',
  'salary'),
 ('0.38', '0.53', '2', '157', '3', '0', '1', '0', 'sales', 'low'),
 ('0.8', '0.86', '5', '262', '6', '0', '1', '0', 'sales', 'medium'),
 ('0.11', '0.88', '7', '272', '4', '0', '1', '0', 'sales', 'medium'),
 ('0.72', '0.87', '5', '223', '5', '0', '1', '0', 'sales', 'low')]

In [24]:
hr_kaggle_final1 = hr_kaggle_final.toDF(['Satisfaction_Level', 'Last_Evaluation', 'no_of_projects', 'Average_hours', 'Time_Spent', 'Work_accident','left', 'Promotion', 'Dept', 'Salary'])

Py4JJavaError: An error occurred while calling o23.applySchemaToPythonRDD.
: java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
	at org.apache.hadoop.hive.ql.session.SessionState.start(SessionState.java:522)
	at org.apache.spark.sql.hive.client.HiveClientImpl.<init>(HiveClientImpl.scala:189)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.spark.sql.hive.client.IsolatedClientLoader.createClient(IsolatedClientLoader.scala:258)
	at org.apache.spark.sql.hive.HiveUtils$.newClientForMetadata(HiveUtils.scala:359)
	at org.apache.spark.sql.hive.HiveUtils$.newClientForMetadata(HiveUtils.scala:263)
	at o

In [12]:
indexer = StringIndexer(inputCol="Dept", outputCol="dept")
indexed = indexer.fit(hr_kaggle_final).transform(hr_kaggle_final)

AttributeError: 'PipelinedRDD' object has no attribute '_jdf'